# No damping oscillation of helicopter motion from hover

The following scripts runs a simulation of helicopter motion without damping. The first cell gets the data and all other cells create relevant plots

In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# INITIAL DATA HELICOPTER
g = 9.81
cla = 5.7  # NACA 0012
volh = .075  # blade solidity
lok = 6
cds = 1.5
mass = 2200
rho = 1.225
vtip = 200
diam = 2 * 7.32
iy = 10615
mast = 1
omega = vtip / (diam / 2)
area = np.pi / 4 * diam ** 2
tau = .1  # time constant in dynamics inflow!!!
collect = [6 * np.pi / 180]
longit = [0 * np.pi / 180]

# INITIAL VALUES SIMULATION
t0 = 0
u0 = 0
w0 = 0
q0 = 0
pitch0 = 0 * np.pi / 180
x0 = 0
labi0 = np.sqrt(mass * g / (area * 2 * rho)) / vtip  # nondimensional inflow

t = [t0]
u = [u0]
w = [w0]
q = [q0]
pitch = [pitch0]
x = [x0]
labi = [labi0]
z = [0]

# START INTEGRATION FOR 40 SECONDS
aantal = 400
teind = 40
stap = (teind - t0) / aantal

for i in range(aantal):
    if t[i] >= 0.5 and t[i] <= 1:
        longit.append(1 * np.pi / 180)  # longit cyclic input
    else:
        longit.append(0 * np.pi / 180)
    
    # no LAW FOR COLLECTIVE
    c = u[i] * np.sin(pitch[i]) - w[i] * np.cos(pitch[i])
    h = -z[i]
    collect.append(collect[0])    

    # Defining the differential equations
    # defining the nondimensional notations
    qdiml = q[i] / omega
    vdiml = np.sqrt(u[i]**2 + w[i]**2) / vtip
    if u[i] == 0:
        phi = np.pi/2 if w[i] > 0 else -np.pi/2
    else:
        phi = np.arctan(w[i] / u[i])
    if u[i] < 0:
        phi += np.pi
    alfc = longit[i] - phi

    mu = vdiml * np.cos(alfc)
    labc = vdiml * np.sin(alfc)

    # a1 Flapping calculus
    teller = -16/lok*qdiml + 8/3*mu*collect[i] - 2*mu*(labc + labi[i])
    a1 = teller / (1 - .5*mu**2)

    # the thrust coefficient
    ctelem = cla*volh/4*(2/3*collect[i]*(1+1.5*mu**2)-(labc+labi[i]))
    # Thrust coefficient from Glauert
    alfd = alfc - a1
    ctglau = 2*labi[i]*np.sqrt((vdiml*np.cos(alfd))**2 + (vdiml*np.sin(alfd) + labi[i])**2)

    # Equations of motion
    labidot = ctelem
    thrust = labidot * rho * vtip**2 * area
    helling = longit[i] - a1
    vv = vdiml * vtip  # it is 1/sqrt(u^2+w^2)

    udot = -g*np.sin(pitch[i]) - cds/mass * .5 * rho * u[i] * vv + thrust/mass * np.sin(helling) - q[i] * w[i]
    wdot = g*np.cos(pitch[i]) - cds/mass * .5 * rho * w[i] * vv - thrust/mass * np.cos(helling) + q[i] * u[i]
    qdot = -thrust * mast / iy * np.sin(helling)
    pitchdot = q[i]
    xdot = u[i]
    zdot = -c

    labidot = (ctelem - ctglau) / tau

    # Euler integration
    u.append(u[i] + stap * udot)
    w.append(w[i] + stap * wdot)
    q.append(q[i] + stap * qdot)
    pitch.append(pitch[i] + stap * pitchdot)
    x.append(x[i] + stap * xdot)
    labi.append(labi[i] + stap * labidot)
    z.append(z[i] + stap * zdot)
    t.append(t[i] + stap)


# Let us now plot the data


In [ ]:

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=np.array(longit) * 180 / np.pi, mode='lines', name='longit (deg)'))
fig.update_layout(title='Longitudinal Input over Time',
                  xaxis_title='t (s)',
                  yaxis_title='longit (deg)',
                  )
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=u, mode='lines', name='u(m/s)'))
fig.add_trace(go.Scatter(x=t, y=np.array(pitch) * 180 / np.pi, mode='lines', name='pitch(deg)'))
fig.update_layout(title='Velocity and Pitch over Time',
                  xaxis_title='t (s)',
                  )
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=x, mode='lines', name='x(m)'))
fig.update_layout(title='Displacement in X-Direction over Time',
                  xaxis_title='t (s)',
                  yaxis_title='x(m)',
                  )
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=w, mode='lines', name='w(m/s)'))
fig.update_layout(title='Vertical Velocity over Time',
                  xaxis_title='t (s)',
                  yaxis_title='w(m/s)',
                  )
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=q, mode='lines', name='q(rad/s)'))
fig.update_layout(title='Rate of Pitch Change over Time',
                  xaxis_title='t (s)',
                  yaxis_title='q(rad/s)',
                  )
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=labi, mode='lines', name='labi'))
fig.update_layout(title='Non-Dimensional Inflow over Time',
                  xaxis_title='t (s)',
                  yaxis_title='labi',
                  )
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=-np.array(z), mode='lines', name='h(m)'))
fig.update_layout(title='Altitude over Time',
                  xaxis_title='t (s)',
                  yaxis_title='h(m)',
                  )
fig.show()